In [1]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.backends import default_backend
import base64

def gen_ecc_account():
    # 生成椭圆曲线密钥对
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())

    # 将私钥保存到文件
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    with open('private_key.pem', 'wb') as f:
        f.write(private_pem)

    # 将公钥保存到文件
    public_pem = private_key.public_key().public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    with open('public_key.pem', 'wb') as f:
        f.write(public_pem)

    return private_key, private_pem, public_pem

def sign_message(private_key, message):
    # 计算消息的哈希值
    message_hash = hashes.Hash(hashes.SHA256(), backend=default_backend())
    message_hash.update(message.encode())
    digest = message_hash.finalize()

    # 签名
    signature = private_key.sign(digest, ec.ECDSA(hashes.SHA256()))

    # 将签名结果编码为Base64
    return base64.b64encode(signature).decode()

if __name__ == '__main__':
    message = "Hello, world!"

    # 生成密钥对
    private_key, private_pem, public_pem = gen_ecc_account()

    # 签名消息
    signature = sign_message(private_key, message)
    print(f"Signature: {signature}")

    # 输出公钥
    print(f"Public Key (PEM): {public_pem.decode()}")

Signature: MEYCIQDl7Decu/FlZStia6Pl92lq0yr8U2WW0CwTLDZGnkkvRQIhAJk1C3M5VJMf/G7W1EHJfayZCCeOHxz4kWXPtxbbAi6T
Public Key (PEM): -----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEn5psdrZIEmbbkzhytkO7TrjTJBHg
wGVyREVQrGQhylEDZq32NFBb65q1mBW8+o1C9ES40zMAN9h7slRp1Lhfyw==
-----END PUBLIC KEY-----



In [2]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes as crypto_hashes
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.backends import default_backend
import base64
import os

def gen_ecc_account(password: str):
    # 生成椭圆曲线密钥对
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())

    # 使用密码加密私钥
    encrypted_private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.BestAvailableEncryption(password.encode())
    )
    with open('encrypted_private_key.pem', 'wb') as f:
        f.write(encrypted_private_pem)

    # 将公钥保存到文件
    public_pem = private_key.public_key().public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    with open('public_key.pem', 'wb') as f:
        f.write(public_pem)

    return private_key, encrypted_private_pem, public_pem

def load_encrypted_private_key(pem_data: bytes, password: str):
    # 从加密的PEM数据加载私钥
    private_key = serialization.load_pem_private_key(
        pem_data,
        password=password.encode(),
        backend=default_backend()
    )
    return private_key

def sign_message(private_key, message: str):
    # 计算消息的哈希值
    message_hash = hashes.Hash(hashes.SHA256(), backend=default_backend())
    message_hash.update(message.encode())
    digest = message_hash.finalize()

    # 签名
    signature = private_key.sign(digest, ec.ECDSA(hashes.SHA256()))

    # 将签名结果编码为Base64
    return base64.b64encode(signature).decode()

if __name__ == '__main__':
    message = "Hello, world!"
    password = "123"

    # 生成密钥对
    private_key, encrypted_private_pem, public_pem = gen_ecc_account(password)

    # 从加密的PEM数据加载私钥
    private_key = load_encrypted_private_key(encrypted_private_pem, password)

    # 签名消息
    signature = sign_message(private_key, message)
    print(f"Signature: {signature}")

    # 输出公钥
    print(f"Public Key (PEM): {public_pem.decode()}")

Signature: MEYCIQCX15FlejF9FhXgwRLzF6jzS8MmXn5cF+oSCELxBCgvNgIhAKSVtw5qlYYt6UFACgepnOoeVY9K2PpBiiXYXVSY6oQV
Public Key (PEM): -----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEw+QkyLUk5BHEPOgevUcm1N6KVIQ5
MmmkjdI7GH1LXkdZo/egrUaYDBbRgQRiZTtVKrEmNfs+dH0fpec40nFTNA==
-----END PUBLIC KEY-----

